CHAPTER 5

In [1]:
import pandas as pd

In [2]:
path = "data/nyc-parking-violations-2020.csv"
df = pd.read_csv(path, usecols=["Plate ID", "Registration State", "Vehicle Make", "Vehicle Color", "Violation Time", "Street Name"])

In [5]:
# rows total
total = len(df.index)
total

12495734

In [6]:
# rows without any nans
without_any_nans = len(df.dropna().index)
without_any_nans

12048375

In [7]:
# diff with 100$ cost
(total - without_any_nans) * 100


44735900

In [9]:
# loss with subset features
subset = ["Plate ID", "Registration State", "Vehicle Make", "Street Name"]
print(f'Loss: {(len(df) - len(df.dropna(subset=subset))) * 100}')

subset = ["Plate ID", "Registration State", "Street Name"]
print(f'Loss without car make: {(len(df.index) - len(df.dropna(subset=subset).index)) * 100}')

Loss: 6378500
Loss without car make: 161800


In [20]:
'''
How many rows would you eliminate if you required at least three non-null values
from the four columns Plate ID, Registration State, Vehicle Make, and
Street Name
'''

rows_with_at_least_3_non_nans = len(
    df[
        (
            df['Plate ID'].notnull().astype(int) + 
            df['Registration State'].notnull().astype(int) + 
            df['Vehicle Make'].notnull().astype(int) + 
            df['Street Name'].notnull().astype(int)
        ) >= 3
    ]
)
total - rows_with_at_least_3_non_nans

253

In [25]:
'''Which of the columns you've imported has the greatest number of NaN values'''
df.isnull().astype(int).sum()

Plate ID                 202
Registration State         0
Vehicle Make           62420
Violation Time           278
Street Name             1417
Vehicle Color         391982
dtype: int64

In [26]:
'''
Null data is bad, but there is plenty of bad non-null data, too. For example,
many cars with BLANKPLATE as a plate ID were ticketed. Turn these into NaN val-
ues, and rerun the previous query.
'''
df['Plate ID'] = df['Plate ID'].replace('BLANKPLATE', pd.NA)
df.isnull().astype(int).sum()

Plate ID                9084
Registration State         0
Vehicle Make           62420
Violation Time           278
Street Name             1417
Vehicle Color         391982
dtype: int64

In [8]:
df = None